# Object Classification from 3D-Point Cloud data

In this tutorial, we are going to train (Hands-on) the PointNet on ModelNet40 PointCloud Dataset for object classification in 3D. 

All the theoretical intuitions are provided in the [Introduction Notebook](0-Introduction.ipynb).

In [ ]:
### import required modules
import os
import torch
import numpy as np
from tqdm import tqdm
from dataloaders.ModelNetDataLoader import ModelNetDataLoader
from utilities.data_manipulation import random_point_dropout, random_scale_point_cloud, shift_point_cloud  # The test function
from utilities.activation import inplace_relu    # To save memory

In the following, we define the number of parameters and their values. 

In [ ]:
class Args:
    '''PARAMETERS'''
    use_cpu =False
    gpu='0'
    batch_size = 24
    model='pointnet_cls'
    num_category = 40
    epoch=200
    learning_rate=0.001
    num_point=1024
    optimizer='Adam'
    log_dir = 'runs'
    decay_rate=1e-4
    use_normals=False
    process_data=False
    use_uniform_sample=False

args = Args()

In the following block, we define the test function. We will use this function inside our training loop to validate and see the performance of our model.

In [ ]:
def test(model, loader, num_class=args.num_category):
    mean_correct = []
    class_acc = np.zeros((num_class, 3))
    classifier = model.eval()

    for j, (points, target) in tqdm(enumerate(loader), total=len(loader)):

        if not args.use_cpu:
            points, target = points.cuda(), target.cuda()

        points = points.transpose(2, 1)
        pred, _ = classifier(points)
        pred_choice = pred.data.max(1)[1]

        for cat in np.unique(target.cpu()):
            classacc = pred_choice[target == cat].eq(target[target == cat].long().data).cpu().sum()
            class_acc[cat, 0] += classacc.item() / float(points[target == cat].size()[0])
            class_acc[cat, 1] += 1

        correct = pred_choice.eq(target.long().data).cpu().sum()
        mean_correct.append(correct.item() / float(points.size()[0]))

    class_acc[:, 2] = class_acc[:, 0] / class_acc[:, 1]
    class_acc = np.mean(class_acc[:, 2])
    instance_acc = np.mean(mean_correct)

    return instance_acc, class_acc

Verify if a Nvidia GPU is available for training the network. Check args.gpu value to define the available GPU in a cluster of GPUs.

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = args.gpu

In the following, we will load the ModelNetX (X=10/40) Dataset from the disk. To download the dataset, follow the download and preparation instructions given in [Introduction Notebook](0-Introduction.ipynb). For this tutorial, we are using ModelNet40 having 40 classes. It is also possible to use ModelNet10 with ten object classes, a subgroup of the ModelNet40 dataset.

In [ ]:
### load the data from the hard disk
data_path = '../data/modelnet40_normal_resampled/'

train_dataset = ModelNetDataLoader(root=data_path, args=args,  split='train', process_data=args.process_data)
test_dataset = ModelNetDataLoader(root=data_path, args=args, split='test', process_data=args.process_data)
trainDataLoader = torch.utils.data.DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, num_workers=10, drop_last=True)
testDataLoader = torch.utils.data.DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False, num_workers=10)


In the following we define the PointNet classification model. To visualize how the model looks like, check [Introduction Notebook](0-Introduction.ipynb).

In [ ]:
num_class = args.num_category    ### number of classes.
from models.pointnet_cls import get_model, get_loss
classifier = get_model(num_class, normal_channel=args.use_normals)
criterion = get_loss()
classifier.apply(inplace_relu)

In the following, we define the model optimizer and we initialize the schedular.

In [ ]:
if args.optimizer == 'Adam':
    optimizer = torch.optim.Adam(
        classifier.parameters(),
        lr=args.learning_rate,
        betas=(0.9, 0.999),
        eps=1e-08,
        weight_decay=args.decay_rate
    )
else:
    optimizer = torch.optim.SGD(classifier.parameters(), lr=0.01, momentum=0.9)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.7)

In the following, we define the path to save the best trained weights.

In [ ]:
if not args.use_cpu:
    classifier = classifier.cuda()
    criterion = criterion.cuda()

try:
    checkpoint = torch.load('save_weights/best_model_classification.pth')
    start_epoch = checkpoint['epoch']
    classifier.load_state_dict(checkpoint['model_state_dict'])
    print('Use pretrain model')
except:
    print('No existing model, starting training from scratch...')
    start_epoch = 0


In the following block, we define and initialize the parameters for performance.

In [ ]:
global_epoch = 0
global_step = 0
best_instance_acc = 0.0
best_class_acc = 0.0

#### ***Following is the training loop.***

In [ ]:
print('Start training...')
for epoch in range(start_epoch, args.epoch):
    print('Epoch %d (%d/%s):' % (global_epoch + 1, epoch + 1, args.epoch))
    mean_correct = []
    classifier = classifier.train()

    scheduler.step()
    for batch_id, (points, target) in tqdm(enumerate(trainDataLoader, 0), total=len(trainDataLoader), smoothing=0.9):
        optimizer.zero_grad()

        points = points.data.numpy()
        points = random_point_dropout(points)
        points[:, :, 0:3] = random_scale_point_cloud(points[:, :, 0:3])
        points[:, :, 0:3] = shift_point_cloud(points[:, :, 0:3])
        points = torch.Tensor(points)
        points = points.transpose(2, 1)

        if not args.use_cpu:
            points, target = points.cuda(), target.cuda()

        pred, trans_feat = classifier(points)
        loss = criterion(pred, target.long(), trans_feat)
        pred_choice = pred.data.max(1)[1]

        correct = pred_choice.eq(target.long().data).cpu().sum()
        mean_correct.append(correct.item() / float(points.size()[0]))
        loss.backward()
        optimizer.step()
        global_step += 1

    train_instance_acc = np.mean(mean_correct)
    print('Train Instance Accuracy: %f' % train_instance_acc)

    with torch.no_grad():
        instance_acc, class_acc = test(classifier.eval(), testDataLoader, num_class=num_class)

        if (instance_acc >= best_instance_acc):
            best_instance_acc = instance_acc
            best_epoch = epoch + 1

        if (class_acc >= best_class_acc):
            best_class_acc = class_acc
        print('Test Instance Accuracy: %f, Class Accuracy: %f' % (instance_acc, class_acc))
        print('Best Instance Accuracy: %f, Class Accuracy: %f' % (best_instance_acc, best_class_acc))

        if (instance_acc >= best_instance_acc):
            print('Save model...')
            savepath = 'save_weights/' + 'best_model_classification.pth'
            print('Saving at %s' % savepath)
            state = {
                'epoch': best_epoch,
                'instance_acc': instance_acc,
                'class_acc': class_acc,
                'model_state_dict': classifier.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
            }
            torch.save(state, savepath)
        global_epoch += 1

print('End of training...')
